In [21]:
# basic imports

import numpy as np
import pandas as pd

In [22]:
#importing the dataset

mv = pd.read_csv("tmdb_5000_movies.csv")
cr = pd.read_csv("tmdb_5000_credits.csv")

In [23]:
# #checking the data

# #mv.head(2)
# cr.head(2)

In [24]:
#merging the dataset into one data set using the "title"

movies = mv.merge(cr,on="title")

In [25]:
# #checking the shape of the dataset

# print(mv.shape)
# print(cr.shape)
# movies.shape

In [26]:
# #checking the merging

# movies.head(2)

# feature selection 

In [27]:
# # to get the info about all the features

# movies.info()

## feature selection 
- genres
- id (for fetching the poster show)
- keywords
- title (not original title, can be in regional language)
- overview
- cast
- crew

- for afterwards
     - release date


In [28]:
#selecting which features we need! and store it into the same name, (can use diff.)

movies = movies[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [29]:
# #just checking 

# movies.head(2) 

In [30]:
# #checking for the missing data

# movies.isnull().sum()

In [31]:
#drop the missing data

movies.dropna(inplace = True)

In [32]:
# movies.isnull().sum()

In [33]:
# #checking for any duplicate data

# movies.duplicated().sum()

In [34]:
# #checking the format of the genres for the iloc[0]

# movies.iloc[0].genres 

- this is a list of dict
    - convert this string of list into list
        - using 'ast' in python

In [35]:
import ast
# ast.literal_eval

def convert(obj):
    l =[]
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l

In [36]:
#storing that into the same col

movies["genres"] = movies["genres"].apply(convert)
movies["keywords"] = movies["keywords"].apply(convert)

In [37]:
# movies.head(2)

In [38]:
# def convert_3(obj):
#     l = []
#     cou = 0
#     for i in ast.literal_eval(obj):
#         if cou != 3:
#             l.append(i["name"])
#             cou += 1
#         else:
#             break
#     return l

# movies["cast"] = movies["cast"].apply(convert_3)

In [39]:
#convert into a string

def convert_3(obj):
    l = []
    cou = 0
    for i in ast.literal_eval(obj):
        if cou != 3:
            l.append(i["name"])
            cou += 1
        else:
            break
    return str(l)

In [40]:
# # def get_first_3_cast_members(cast_str):
# #     return cast_str.strip("[]").replace("'", "").split(", ")[:3]

# movies["cast"] = movies["cast"].apply(convert_3).apply(get_first_3_cast_members)

In [41]:
# movies.head(2)

In [42]:
def fetch_director(obj):
    l =[]
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
    return l

In [43]:
movies["crew"] = movies["crew"].apply(fetch_director)

In [44]:
# movies.head()

In [45]:
# movies["overview"][3]

### so the "overview" column is a string
- have to convert it into a string
- so that it can be concatinate with others

In [46]:
movies["overview"] = movies["overview"].apply(lambda x:x.split())

## we have to transform the last columns 
- so that no spaces between them
    - because there can be same name person 
    - or same surname person , can be confusing

In [47]:
movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ","") for i in x])

movies["crew"] = movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])

In [48]:
# movies.head()

In [49]:
# movies["cast"][23]

In [50]:
# # movies["cast"] = movies["cast"].apply(lambda x:[i.replace(" ","") for i in x])

# movies["cast"] = movies["cast"].apply(lambda x: ["".join(i.split()) for i in x])

for index, row in movies.iterrows():
    new_cast = []
    for actor in row["cast"]:
        new_cast.append(actor.replace(" ", ""))
    movies.at[index, "cast"] = new_cast


In [51]:
# movies.head(2)

In [52]:
movies["tags"] = movies["overview"] +  movies["genres"] + movies["keywords"] +  movies["cast"] +  movies["crew"]

In [53]:
# movies.head(2)

In [54]:
movie = movies[["movie_id","title","tags"]]

In [55]:
# movie.head()

In [56]:
movie["tags"].apply(lambda x:" ".join(x))

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: tags, Length: 4806, dtype: object

In [57]:
movie["tags"] = movie["tags"].apply(lambda x:" ".join(x)) #with warning
#movie.loc[:, "tags"] = movie["tags"].apply(lambda x: " ".join(x))


/tmp/ipykernel_3935146/750453074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie["tags"] = movie["tags"].apply(lambda x:" ".join(x)) #with warning


In [58]:
# movie.head()

# THE FIRST PART IS READY NOW

## Text to Vectors
[`link for further study`](https://www.deepset.ai/blog/what-is-text-vectorization-in-nlp)


- ### Count-Based Text Vectorization

    - In programming, a vector is a data structure that is like a list or an array. For input representation, it is a set of values, with the number of values representing the vector’s “dimensionality.” Vectors contain information about the qualities of an input object. They offer a format that computers can easily process.

- #### Bag of Words

    - A simple vectorization method for text is a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) (BoW) representation. A BoW vector has the length of the entire vocabulary — that is, the set of unique words in the corpus. The vector’s values represent the frequency with which each word appears in a given text passage.
    
- #### TF-IDF

    - Weighted BoW text vectorization techniques like [TF-IDF](https://haystack.deepset.ai/components/retriever#tf-idf) (short for “term frequency-inverse document frequency), attempt to give higher relevance scores to words that occur in fewer documents within the corpus. To that end, TF-IDF measures the frequency of a word in a text against its overall frequency in the corpus.

- #### BM25

    - While more sophisticated than the simple BoW approach, [BM25](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables) was introduced to address some of its shortcomings. It is an improvement over TF-IDF and takes into account the length of the document. It also dampens the effect of having many occurrences of a word in a document.

    - BoW methods produce long vectors that contain many zeros, so they’re often called “sparse.” In addition to being language-independent, sparse vectors are quick to compute and compare. [Semantic search](https://www.deepset.ai/blog/understanding-semantic-search) systems use them for quick document retrieval.

In [59]:
# movie["tags"][3] #for checking 

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")

In [61]:
vectors = cv.fit_transform(movie["tags"]).toarray()

In [62]:
# vectors.shape

In [63]:
# vectors

In [64]:
# cv.get_feature_names()

In [65]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [66]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [67]:
# ps.stem("Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy. Action Crime Drama Thriller dccomics crimefighter terrorist secretidentity burglar hostagedrama timebomb gothamcity vigilante cover-up superhero villainess tragichero terrorism destruction catwoman catburglar imax flood criminalunderworld batman ChristianBale MichaelCaine GaryOldman ChristopherNolan")

In [68]:
movie["tags"] = movie["tags"].apply(stem)

/tmp/ipykernel_3935146/1092719389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie["tags"] = movie["tags"].apply(stem)


In [69]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [70]:
from sklearn.metrics.pairwise import cosine_similarity

In [71]:
similarity = cosine_similarity(vectors)

In [72]:
sorted(list(enumerate(similarity[0])),reverse = True,key=lambda x:x[1])[1:6]

[(539, 0.26940795304016235),
 (1194, 0.2545875386086578),
 (507, 0.25182770057259657),
 (1216, 0.24459979523511427),
 (260, 0.2434322477800738)]

In [73]:
# similarity[34]

In [74]:
def recommend(movies):
    movie_index = movie[movie["title"] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        #print(i[0])
        print(movie.iloc[i[0]].title)

In [77]:
# user_input = input("Enter the Movie name: ")
# recommend ("user_input")

## end of the algorithm